# 功能:掃指定範圍的所有ema均線組合,找出獲利/勝率最大的組合.
* 進場:均線黃金交叉
* 出場:均線死忘交叉 或 收盤價低於sma扺扣價
* 方向:多,空,多空雙向

### 範圍開太大會跑很久
### 開啟google drive是為了把結果存檔到CSV檔.不然colab會自動消失.不想用的請自註解掉.

In [1]:
#戴入google drive
from google.colab import drive
drive.mount('/content/drive') # 此處需要登入google帳號


Mounted at /content/drive


In [2]:
try:
    import vectorbt
except ModuleNotFoundError as e:
    !pip install vectorbt    

try:
    import yfinance
except ModuleNotFoundError as e:
    !pip install yfinance   

try:
    import ccxt
except ModuleNotFoundError as e:
    !pip install ccxt

try:
    import talib
except ModuleNotFoundError as e:
    !pip install talib_binary         


from IPython.display import clear_output
clear_output()

!exit

In [3]:

import vectorbt as vbt
import talib
import pandas as pd


代號名和資料來源是有相關性的.
*   CCXT:應該只支援加密貨幣.ex: BTC/USDT
*   Yahoo_Finance:台股,美股,加密貨幣都支援. ex: 2357.tw, TSLA, BTC-USD
(https://finance.yahoo.com/)

timeframe和資料來源也是有相關性的.
*   CCXT: ['1m', '5m', '15m', '1h', '4h', '1d', '3d']
*   Yahoo_finance: ['1m', '2m', '5m', '15m', '30m', '60', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo']



In [4]:
#@title 回測設定
symbol_name = "TSLA" #@param {type:"string"}
data_source = "Yahoo_Finance" #@param ["CCXT","Yahoo_Finance"]

#ccxt timeframe = ['1m', '5m', '15m', '1h', '4h', '1d', '3d']
timeframe_list = ['1h','1d']

date_start = "2021-01-01" #@param {type:"date"}
date_end = "2022-09-21" #@param {type:"date"}

ema_s_start = 8 #@param {type:"integer"}
ema_s_end = 25 #@param {type:"integer"}

ema_m_start = 12 #@param {type:"integer"}
ema_m_end = 69 #@param {type:"integer"}

sma_deduct_start = 5 #@param {type:"integer"}
sma_deduct_end = 30 #@param {type:"integer"}


init_cash = 100 #@param {type:"integer"}

direction = "both" #@param ["longonly","shortonly","both"]
#@markdown 多工平行運算(可減少25%時間)
multi_processing = True #@param {type:"boolean"}



ema_s_list = [ema for ema in range(ema_s_start, ema_s_end+1)]
ema_m_list = [ema for ema in range(ema_m_start, ema_m_end+1)]
sma_d_list = [0]+[d for d in range(sma_deduct_start,sma_deduct_end+1)]

In [5]:
#download data

ohlc_data = dict()
if data_source == "CCXT":
    for i in timeframe_list:
        data = vbt.CCXTData.download(
            symbols=symbol_name,
            start=date_start,
            end=date_end,
            timeframe=i
        )
        ohlc_data.update(
            {i: data}
        )
elif data_source == "Yahoo_Finance":
    for i in timeframe_list:
        data = vbt.YFData.download(
            symbols=symbol_name,
            start=date_start,
            end=date_end,
            interval=i
        )
        ohlc_data.update(
            {i: data}
        )


In [6]:
#define function


def scan_ma(timeframe):

    results = pd.DataFrame()    

    close_price = ohlc_data[timeframe].get("Close")
    
    for i_es in ema_s_list:
        for i_em in ema_m_list:

            ema_sline = talib.EMA(close_price, timeperiod=i_es)
            ema_mline = talib.EMA(close_price, timeperiod=i_em) 

            ema_entry = [False]
            ema_exit = [False]
            sma_d_exit = [False] 

            for i in range(1,ema_sline.shape[0]):
                if ema_sline[i] > ema_mline[i] and ema_sline[i-1] < ema_mline[i-1]:
                    ema_entry.append(True)
                else:
                    ema_entry.append(False)

                if ema_sline[i] < ema_mline[i] and ema_sline[i-1] > ema_mline[i-1]:
                    ema_exit.append(True)
                else:
                    ema_exit.append(False)

                
            for i_sd in sma_d_list:
               
                if i_sd == 0:
                    pf = vbt.Portfolio.from_signals(
                        close=close_price,
                        entries=ema_entry,
                        exits=ema_exit,
                        init_cash=init_cash,
                        direction=direction,
                        
                    )
                else:
                    sma_d_exit=[False]    
                    for i in range(1,close_price.shape[0]):
                        if close_price[i] < close_price[i-i_sd]:
                            sma_d_exit.append(True)
                        else:
                            sma_d_exit.append(False)    

                    pf = vbt.Portfolio.from_signals(
                        close=close_price,
                        entries=ema_entry,
                        exits=sma_d_exit,
                        init_cash=init_cash,
                        direction=direction
                    )   
                temp_series = pd.Series([timeframe,i_es,i_em,i_sd],index=["timeframe","ema_s","ema_m","sma_d"])

                results = pd.concat([results,temp_series.append(pf.stats(silence_warnings=True))],axis=1)


    return results



In [ ]:
#main 

import warnings
#warnings.filterwarnings('ignore')

import multiprocessing as mp

cpus = mp.cpu_count()
pool = mp.Pool(processes=cpus)

result_df = pd.DataFrame()

if multi_processing is True:

    result = pool.map(scan_ma,timeframe_list)

    for i in result:
        result_df = pd.concat([result_df,i],axis=1)
    result_df = result_df.T
    result_df.reset_index(drop=True,inplace=True)
else:
    
    for i in timeframe_list:
        result = scan_ma(i)
        result_df = pd.concat([result_df,result],axis=1,ignore_index=True)
    result_df = result_df.T

print(f"Full combination count: {result_df.shape[0]}")

In [ ]:
#依獲利排序，取前100, 存到google drive ,檔案格式csv
import os

result_df.sort_values("End Value", ascending=False, inplace=True)
result_df.head(100).to_csv(f"{symbol_name.replace('/','')}_endvaluetable.csv")
if os.path.exists("/content/drive/MyDrive/Colab Notebooks/"):
    result_df.head(100).to_csv(f"/content/drive/MyDrive/Colab Notebooks/{symbol_name.replace('/','')}_endvaluetable.csv")
result_df.head(10)


In [ ]:
#依Win Rate排序，取前100, 存到csv檔

result_df.sort_values("Win Rate [%]", ascending=False, inplace=True)
result_df.head(100).to_csv(f"{symbol_name.replace('/','')}_winratetable.csv")
if os.path.exists("/content/drive/MyDrive/Colab Notebooks/"):
    result_df.head(100).to_csv(f"/content/drive/MyDrive/Colab Notebooks/{symbol_name.replace('/','')}_winratetable.csv")
result_df.head(10)